In [1]:
import xml.etree.ElementTree as ET
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input,VGG16


#from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
#from keras.applications.vgg19 import preprocess_input
from keras.models import Model

from sklearn.metrics import precision_recall_fscore_support
from keras import backend as K
from keras.models import load_model
#from keras.models import load_weights




Using TensorFlow backend.


In [2]:

# Precison , Recall
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [3]:
xmlPath = "C:/Users/Parashara Ramesh/Desktop/PESU/AI/hackathon/images/VOCdevkit/VOC2010/Annotations/"
imgPath = "C:/Users/Parashara Ramesh/Desktop/PESU/AI/hackathon/images/VOCdevkit/VOC2010/JPEGImages/"
#imgPath2 = "C:/Users/Parashara Ramesh/Desktop/PESU/AI/hackathon/images/VOCdevkit/VOC2010/JPEGImages/"

labels=["aeroplane","bicycle","bird","boat","bottle","bus","car","cat","chair","cow","diningtable","dog","horse","motorbike","person","pottedplant","sheep","sofa","train","tvmonitor"]

imagewidth=150
imageheight=150
input_shape = (imageheight, imagewidth, 3)

xtrain=[]
ytrain = []

In [4]:
def createYtrain():
    dirs = os.listdir(xmlPath)
    count=0
    for item in dirs[:5000]:
        onehot=[0]*20
        if os.path.isfile(xmlPath + item):
            e = ET.parse(xmlPath + item)
            root = e.getroot()
            obj = root.findall('object')
            tags=[]
            for k in obj:
                tags.append(k[0].text)
            for tag in tags:
                i = labels.index(tag)
                onehot[i]=1
            ytrain.append(onehot)

createYtrain()
ytrain=np.asarray(ytrain)
print("y shape is ",ytrain.shape)




y shape is  (5000, 20)


In [5]:
def createXtrain():
    dirs = os.listdir(imgPath)
    for item in dirs[:5000]:
        if os.path.isfile(imgPath + item):
            image = Image.open(imgPath+item)
            resize = image.resize((imagewidth,imageheight), Image.NEAREST) 
            data = np.asarray(resize)
            #data=np.asarray(image)
            xtrain.append(data)

createXtrain() 
xtrain=np.asarray(xtrain)
print("xshape is ",xtrain.shape)

xshape is  (5000, 150, 150, 3)


In [6]:
def createTrainandTest():
    x_train, x_test, y_train, y_test = train_test_split(xtrain, ytrain, test_size=0.10, random_state=42)
    return x_train,x_test,y_train,y_test
x_train,x_test,y_train,y_test=createTrainandTest()
print("created test and train data")

created test and train data


In [7]:

model = Sequential()

#Conv2D parameters : Conv2D(32,(3,3), input_shape)
#32 is the output shape
#(3,3) is the filter size of the window in CNN
#input_shape is input shape as said on the forums
model.add(Conv2D(32,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2))) #reducing pixels by 2,2

#one more cnn layer
model.add(Conv2D(32,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

#one more cnn layer
model.add(Conv2D(64,(3,3),input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64)) #64 units underneath each CNN model
model.add(Activation('relu'))
model.add(Dropout(0.5)) # alpha value that is the rate at which the training algorithm has to perform
model.add(Dense(20)) #output layer is 1 i smy assumption
#make dense 20 to get 20 classes as output
#model.add(Activation('softmax'))
model.add(Activation('sigmoid'))
'''

base_model=load_model('vgg16MODEL.h5')
#base_model.save('vgg16MODEL.h5')
print("loaded vgg16")
model = Sequential()
vggmodel=base_model.layers[:-1]
for l in vggmodel:
    model.add(l)
#model.add(Flatten())
model.add(Dense(20))
model.add(Activation('sigmoid'))
#model.add(Dropout(0.5))
'''
print("now added")
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])#,precision,recall])
print(model.summary())
print("now compiling")

now added
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     


In [8]:
model.fit(xtrain, ytrain, epochs=18,batch=8)

Epoch 1/18
5000/5000 [==============================] - 16s - loss: 2.7680 - acc: 0.8155    
Epoch 2/18
5000/5000 [==============================] - 10s - loss: 0.3450 - acc: 0.9007    
Epoch 3/18
5000/5000 [==============================] - 10s - loss: 0.2889 - acc: 0.9142    
Epoch 4/18
5000/5000 [==============================] - 10s - loss: 0.2777 - acc: 0.9181    
Epoch 5/18
5000/5000 [==============================] - 10s - loss: 0.2673 - acc: 0.9212    
Epoch 6/18
5000/5000 [==============================] - 10s - loss: 0.2590 - acc: 0.9234    
Epoch 7/18
5000/5000 [==============================] - 10s - loss: 0.2538 - acc: 0.9251    
Epoch 8/18
5000/5000 [==============================] - 10s - loss: 0.2456 - acc: 0.9269    
Epoch 9/18
5000/5000 [==============================] - 10s - loss: 0.2410 - acc: 0.9272    
Epoch 10/18
5000/5000 [==============================] - 10s - loss: 0.2374 - acc: 0.9283    
Epoch 11/18
5000/5000 [==============================] - 10s - loss: 

In [9]:
score = model.evaluate(x_test,y_test,batch_size=2)
print(score)

print("now saving the model")
model.save('objdetector5096.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

print("finished saving now try testing!")
# returns a compiled model
# identical to the previous one

490/500 [============================>.] - ETA: 0s[0.14608442753925918, 0.94569999051094056]
now saving the model
finished saving now try testing!
